In [1]:
import pandas as pd
import fasttext
import pymorphy3
from multiprocessing import Pool
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
import re

In [2]:
lenta = pd.read_csv('lenta-ru-news-part.csv', usecols=['title', 'text', 'topic'])
lenta.head()

,title,text,topic
0,Телеканалы станут вещать по единому тарифу,С 1 января 2000 года все телеканалы будут опла...,Экономика
1,"Volkswagen выкупает остатки акций ""Шкоды""",Германский автопромышленный концерн Volkswagen...,Экономика
2,Прибыль Тюменнефтегаза возросла в 10 раз,"Нераспределенная прибыль ОАО ""Тюменнефтегаз"", ...",Экономика
3,Крупнейшее в истории слияние компаний происход...,Две крупнейших телекоммуникационных компании С...,Экономика
4,ГАЗ получил четверть обещанного кредита,"ОАО ""ГАЗ"" и Нижегородский банк Сбербанка Росси...",Экономика


In [3]:
lenta.shape

(258273, 3)

In [4]:
lenta.topic.value_counts() # таргет задачи

topic
Экономика          79528
Спорт              64413
Культура           53797
Наука и техника    53136
Бизнес              7399
Name: count, dtype: int64

In [6]:
lenta['title'].str.lower()

0                телеканалы станут вещать по единому тарифу
1                 volkswagen выкупает остатки акций "шкоды"
2                  прибыль тюменнефтегаза возросла в 10 раз
3         крупнейшее в истории слияние компаний происход...
4                   газ получил четверть обещанного кредита
                                ...                        
258268    российский боец ufc включен в книгу рекордов г...
258269    бывший чемпион ufc не выдержал кровопролития и...
258270               моуринью сравнил футболистов с мебелью
258271          путин предостерег от запретов рэп-концертов
258272    падение горнолыжника на полной скорости попало...
Name: title, Length: 258273, dtype: object

In [7]:
m = pymorphy3.MorphAnalyzer()

regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text)
    except:
        return []

In [8]:
def lemmatize(text, pymorphy=m):
    try:
        return " ".join([pymorphy.parse(w)[0].normal_form for w in text])
    except:
        return " "

In [9]:
def clean_text(text):
    return lemmatize(words_only(text))

In [10]:
with Pool(8) as p:
    lemmas = list(tqdm(p.imap(clean_text, lenta['title']), total=len(lenta)))
    
lenta['lemmas'] = lemmas
lenta.head()

/tmp/ipykernel_11443/1996037650.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  lemmas = list(tqdm(p.imap(clean_text, lenta['title']), total=len(lenta)))


  0%|          | 0/258273 [00:00<?, ?it/s]

,title,text,topic,lemmas
0,Телеканалы станут вещать по единому тарифу,С 1 января 2000 года все телеканалы будут опла...,Экономика,телеканал стать вещать по единый тариф
1,"Volkswagen выкупает остатки акций ""Шкоды""",Германский автопромышленный концерн Volkswagen...,Экономика,volkswagen выкупать остаток акция шкода
2,Прибыль Тюменнефтегаза возросла в 10 раз,"Нераспределенная прибыль ОАО ""Тюменнефтегаз"", ...",Экономика,прибыль тюменнефтегаз возрасти в раз
3,Крупнейшее в истории слияние компаний происход...,Две крупнейших телекоммуникационных компании С...,Экономика,крупный в история слияние компания происходить...
4,ГАЗ получил четверть обещанного кредита,"ОАО ""ГАЗ"" и Нижегородский банк Сбербанка Росси...",Экономика,газ получить четверть обещать кредит


In [11]:
with Pool(8) as p:
    lemmas = list(tqdm(p.imap(clean_text, lenta['topic']), total=len(lenta)))

lenta['topic_lem'] = lemmas
lenta.head() 

/tmp/ipykernel_11443/3375837497.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  lemmas = list(tqdm(p.imap(clean_text, lenta['topic']), total=len(lenta)))


  0%|          | 0/258273 [00:00<?, ?it/s]

,title,text,topic,lemmas,topic_lem
0,Телеканалы станут вещать по единому тарифу,С 1 января 2000 года все телеканалы будут опла...,Экономика,телеканал стать вещать по единый тариф,экономика
1,"Volkswagen выкупает остатки акций ""Шкоды""",Германский автопромышленный концерн Volkswagen...,Экономика,volkswagen выкупать остаток акция шкода,экономика
2,Прибыль Тюменнефтегаза возросла в 10 раз,"Нераспределенная прибыль ОАО ""Тюменнефтегаз"", ...",Экономика,прибыль тюменнефтегаз возрасти в раз,экономика
3,Крупнейшее в истории слияние компаний происход...,Две крупнейших телекоммуникационных компании С...,Экономика,крупный в история слияние компания происходить...,экономика
4,ГАЗ получил четверть обещанного кредита,"ОАО ""ГАЗ"" и Нижегородский банк Сбербанка Росси...",Экономика,газ получить четверть обещать кредит,экономика


In [12]:
X = lenta.lemmas.tolist()
y = lenta.topic_lem.tolist()

X, y = np.array(X), np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33)
print ("total train examples %s" % len(y_train))
print ("total test examples %s" % len(y_test))

total train examples 173042
total test examples 85231


In [13]:
with open('data.train.txt', 'w+') as outfile:
    for i in range(len(X_train)):
        outfile.write('__label__' + y_train[i] + ' '+ X_train[i] + '\n')

with open('test.txt', 'w+') as outfile:
    for i in range(len(X_test)):
        outfile.write('__label__' + y_test[i] + ' ' + X_test[i] + '\n')

In [14]:
classifier = fasttext.train_supervised('data.train.txt')
result = classifier.test('test.txt')

Read 1M words
Number of words:  43726
Number of labels: 5
Progress: 100.0% words/sec/thread:  189384 lr:  0.000000 avg.loss:  0.137932 ETA:   0h 0m 0s


In [15]:
dir(fasttext)

['BOW',
 'EOS',
 'EOW',
 'FastText',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'absolute_import',
 'cbow',
 'division',
 'load_model',
 'print_function',
 'skipgram',
 'supervised',
 'tokenize',
 'train_supervised',
 'train_unsupervised',
 'unicode_literals']

In [16]:
result

(85231, 0.9585244805293849, 0.9585244805293849)

In [17]:
print('P@1:', result[1])
print('R@1:', result[2])
print('Number of examples:', result[0])

P@1: 0.9585244805293849
R@1: 0.9585244805293849
Number of examples: 85231


In [18]:
X_test

array(['германия начало расследовать деятельность второй лихтенштейнский банк',
       'сми узнать о личный распоряжение мединский перенести балет серебренников',
       'на съёмка пират карибский море мужчина угрожать нож охрана', ...,
       'в финал мировой лига россиянин проиграть',
       'фанат предложить игрок сборная гана свой внучка',
       'лидер баскетбольный цска выбыть из строй до конец год'],
      dtype='<U119')

In [19]:
!head -n 10 test.txt

__label__экономика германия начало расследовать деятельность второй лихтенштейнский банк
__label__культура сми узнать о личный распоряжение мединский перенести балет серебренников
__label__культура на съёмка пират карибский море мужчина угрожать нож охрана
__label__экономика экс президент unicredit обвинить в финансовый махинация
__label__экономика газпром вернуть газ ереван
__label__наука и техника подсчёт калория для диета и спорт назвать бессмысленный
__label__бизнес роспотребнадзор оспорить разрешение суд на торговля санкционный продукт
__label__спорт реал оценить руни и джеррард в миллион фунт
__label__спорт михаил южный выйти в полуфинал турнир в швейцария
__label__бизнес gq выдвинуть сооснователь mail ru group на получение премия человек год


In [20]:
!head -n 10 data.train.txt

__label__культура создатель ходячий мертвец взяться за сериал о изгнание бес
__label__спорт жить в сша олимпийский чемпионка получить российский паспорт
__label__культура музыкант из blink попасть в больница с тромб и стафилококк
__label__спорт лондон включаться в борьба за право проведение олимпиада год
__label__наука и техника фотокамера научить обходиться один фотон на пиксель
__label__спорт семь футболист отказаться играть за сборная камерун на кубок африка
__label__экономика россия определить цена газ для белоруссия в первый квартал год
__label__культура фильм с звезда малефисент попасть в конкурс кинофестиваль в карлов вар
__label__экономика судебный пристав выставить на продажа более четверть акция вымпелком
__label__культура умереть анатолий кубацкий


In [21]:
classifier.predict("лидер crowded house записать альбом с участник radiohead")

(('__label__культура',), array([0.99998808]))

In [22]:
lenta.head()

,title,text,topic,lemmas,topic_lem
0,Телеканалы станут вещать по единому тарифу,С 1 января 2000 года все телеканалы будут опла...,Экономика,телеканал стать вещать по единый тариф,экономика
1,"Volkswagen выкупает остатки акций ""Шкоды""",Германский автопромышленный концерн Volkswagen...,Экономика,volkswagen выкупать остаток акция шкода,экономика
2,Прибыль Тюменнефтегаза возросла в 10 раз,"Нераспределенная прибыль ОАО ""Тюменнефтегаз"", ...",Экономика,прибыль тюменнефтегаз возрасти в раз,экономика
3,Крупнейшее в истории слияние компаний происход...,Две крупнейших телекоммуникационных компании С...,Экономика,крупный в история слияние компания происходить...,экономика
4,ГАЗ получил четверть обещанного кредита,"ОАО ""ГАЗ"" и Нижегородский банк Сбербанка Росси...",Экономика,газ получить четверть обещать кредит,экономика


In [23]:
with Pool(8) as p:
    lemmas = list(tqdm(p.imap(clean_text, lenta['text']), total=len(lenta)))

lenta['text_lem'] = lemmas
lenta.head() 

/tmp/ipykernel_11443/2301943569.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  lemmas = list(tqdm(p.imap(clean_text, lenta['text']), total=len(lenta)))


  0%|          | 0/258273 [00:00<?, ?it/s]

,title,text,topic,lemmas,topic_lem,text_lem
0,Телеканалы станут вещать по единому тарифу,С 1 января 2000 года все телеканалы будут опла...,Экономика,телеканал стать вещать по единый тариф,экономика,с январь год всё телеканал быть оплачивать усл...
1,"Volkswagen выкупает остатки акций ""Шкоды""",Германский автопромышленный концерн Volkswagen...,Экономика,volkswagen выкупать остаток акция шкода,экономика,германский автопромышленный концерн volkswagen...
2,Прибыль Тюменнефтегаза возросла в 10 раз,"Нераспределенная прибыль ОАО ""Тюменнефтегаз"", ...",Экономика,прибыль тюменнефтегаз возрасти в раз,экономика,нераспределённый прибыль оао тюменнефтегаз доч...
3,Крупнейшее в истории слияние компаний происход...,Две крупнейших телекоммуникационных компании С...,Экономика,крупный в история слияние компания происходить...,экономика,два крупный телекоммуникационный компания сша ...
4,ГАЗ получил четверть обещанного кредита,"ОАО ""ГАЗ"" и Нижегородский банк Сбербанка Росси...",Экономика,газ получить четверть обещать кредит,экономика,оао газ и нижегородский банк сбербанк россия п...


In [24]:
X = lenta.text_lem.tolist()
y = lenta.topic_lem.tolist()

X, y = np.array(X), np.array(y)

In [25]:
# X = lenta.text_lem.tolist()
# y = lenta.topic_lem.tolist()
# 
# X, y = np.array(X), np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33)
print ("total train examples %s" % len(y_train))
print ("total test examples %s" % len(y_test))

total train examples 173042
total test examples 85231


In [26]:
with open('data.train_text.txt', 'w+') as outfile:
    for i in range(len(X_train)):
        outfile.write('__label__' + y_train[i] + ' '+ X_train[i] + '\n')

with open('test.txt', 'w+') as outfile:
    for i in range(len(X_test)):
        outfile.write('__label__' + y_test[i] + ' ' + X_test[i] + '\n')

In [27]:
classifier = fasttext.train_supervised('data.train_text.txt')
result = classifier.test('test.txt')

Read 30M words
Number of words:  315654
Number of labels: 5
Progress: 100.0% words/sec/thread:  596729 lr:  0.000000 avg.loss:  0.103889 ETA:   0h 0m 0s


In [28]:
print('P@1:', result[1])
print('R@1:', result[2])
print('Number of examples:', result[0])

P@1: 0.9719116283981181
R@1: 0.9719116283981181
Number of examples: 85231


In [29]:
X_test

array(['американский учёный создать миниатюрный ядерный батарейка размер который сравнимый с размер небольшой монетка статья учёный опубликовать в журнал journal of applied physics letters and journal of radioanalytical and nuclear chemistry а она краткий изложение приводиться в пресс релиз на сайт университет миссури по слово исследователь схема работа устройство напоминать схема работа радиоизотопный источник энергия в этот источник тепловой энергия радиоактивный распад преобразоваться в электрический в новый устройство радиоактивный элемент разместить в жидкий полупроводник частица возникать в результат радиоактивный распад в это материал приводить к появление в полупроводник электрический ток по слово учёный дать схема не являться новый однако традиционно в подобный батарея использоваться микроскопический решётка из твёрдый полупроводник при это высокоэнергетический частица образоваться при распад с время разрушать решётка снижать качество работа батарея жидкий полупроводник позвол

In [30]:
!head -n 10 test.txt

__label__наука и техника американский учёный создать миниатюрный ядерный батарейка размер который сравнимый с размер небольшой монетка статья учёный опубликовать в журнал journal of applied physics letters and journal of radioanalytical and nuclear chemistry а она краткий изложение приводиться в пресс релиз на сайт университет миссури по слово исследователь схема работа устройство напоминать схема работа радиоизотопный источник энергия в этот источник тепловой энергия радиоактивный распад преобразоваться в электрический в новый устройство радиоактивный элемент разместить в жидкий полупроводник частица возникать в результат радиоактивный распад в это материал приводить к появление в полупроводник электрический ток по слово учёный дать схема не являться новый однако традиционно в подобный батарея использоваться микроскопический решётка из твёрдый полупроводник при это высокоэнергетический частица образоваться при распад с время разрушать решётка снижать качество работа батарея жидкий пол

In [31]:
!head -n 10 data.train_text.txt

__label__культура у группа би в время съёмка клип в испания украсть телефон о это в пятница май сообщать вечерний москва коллектив вести прямой трансляция создание ролик на песня лётчик в время перерыв злоумышленник подойти к оставить на штатив устройство на который вестись запись и унести он при это вор попасть на видео запись доступный в социальный сеть вконтакте преступник уехать с съёмочный площадка на машина задержать он не удаться по слово представитель группа гаджет украсть бывший сотрудник техперсонал местный съёмочный команда который быть уволить незадолго до инцидент
__label__культура британский автор исполнитель эда ширан и американский рэпер фаррелла уильямс стать первый победитель новый музыкальный премия корпорация ввс bbc music awards который вручить декабрь в лондон ширан удостоиться награда в номинация хороший британский артист в аналогичный международный категория хороший исполнитель признать фаррелла уильямс сообщать digital spy кроме тот быть назвать композиция в но

In [32]:
classifier.predict("call of duty: black ops возглавить сводный чарт продажа кино музыка и игра великобритания за год о это сообщить журнал mcv с ссылка на дать британский ассоциация продавец продукция индустрия развлечение (entertainment retailers association) продажа последний часть call of duty составить около миллион копия что позволить игра обогнать даже фильм аватар который разойтись на dvd и blu ray тираж в миллион экземпляр при это аватар являться самый прибыльный фильм в история кинематография принести выручка на сумма более миллиард доллар на третий место расположиться игра fifa от ea она продажа перевалить за отметка миллион копия также среди топ кино музыка и игра год в великобритания оказаться танцевальный игра just dance от ubisoft с тираж около миллион экземпляр который суметь обойти фильм и повелитель буря а также мультфильм вверх стоить отметить что игра попасть в тройка лидер превысить аналогичный показатель свой предшественник так продажа black ops оказаться выше прошлый часть call of duty: modern warfare на тысяча экземпляр а fifa разойтись большой fifa на тысяча копия black ops являться последний седьмой часть серия игра call of duty она разработка заниматься студия treyarch (ответственный за call of duty и world at war) activision выпустить игра в ноябрь год на xbox playstation pc wii ds black ops стать сам быстро продавать игра в история она продажа составить более миллион копия за первый сутки помимо это игра побить рекорд продажа на playstation впервые превысить отметка в миллион экземпляр на этот платформа black ops обогнать аватар по продажа в великобритания")

(('__label__наука',), array([0.96923745]))

In [33]:
lenta.head()

,title,text,topic,lemmas,topic_lem,text_lem
0,Телеканалы станут вещать по единому тарифу,С 1 января 2000 года все телеканалы будут опла...,Экономика,телеканал стать вещать по единый тариф,экономика,с январь год всё телеканал быть оплачивать усл...
1,"Volkswagen выкупает остатки акций ""Шкоды""",Германский автопромышленный концерн Volkswagen...,Экономика,volkswagen выкупать остаток акция шкода,экономика,германский автопромышленный концерн volkswagen...
2,Прибыль Тюменнефтегаза возросла в 10 раз,"Нераспределенная прибыль ОАО ""Тюменнефтегаз"", ...",Экономика,прибыль тюменнефтегаз возрасти в раз,экономика,нераспределённый прибыль оао тюменнефтегаз доч...
3,Крупнейшее в истории слияние компаний происход...,Две крупнейших телекоммуникационных компании С...,Экономика,крупный в история слияние компания происходить...,экономика,два крупный телекоммуникационный компания сша ...
4,ГАЗ получил четверть обещанного кредита,"ОАО ""ГАЗ"" и Нижегородский банк Сбербанка Росси...",Экономика,газ получить четверть обещать кредит,экономика,оао газ и нижегородский банк сбербанк россия п...


In [106]:
X_text = lenta.text_lem.tolist()
X_lemmas = lenta.lemmas.tolist()

In [107]:
X = lenta[['lemmas', 'text_lem']]

In [108]:
y = lenta.topic_lem.tolist()

# X, y = np.array(X), np.array(y)

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33)
print ("total train examples %s" % len(y_train))
print ("total test examples %s" % len(y_test))

total train examples 173042
total test examples 85231


In [98]:
X

array([['телеканал стать вещать по единый тариф',
        'с январь год всё телеканал быть оплачивать услуга передача телерадиосигнал по единый тариф как сообщить министерство российский федерация по дело печать телерадиовещание и средство массовый коммуникация основа для такой решение являться соответствовать постановление правительство в настоящий время общенациональный телеканал (орт нтв вгтрк) оплачивать передача сигнал по льготный тариф который в раз маленький чем у некоторый коммерческий телерадиовещатель с январь для орт нтв и вгтрк тариф быть увеличить в раз для остальной телерадиовещатель снизить'],
       ['volkswagen выкупать остаток акция шкода',
        'германский автопромышленный концерн volkswagen намеренный полностью купить компания skoda как стать сегодня известно из источник в штаб квартира volkswagen в вольфсбург (земля нижний саксония) германский сторона предложить правительство чехиипродать вольфсбург % пакет акция skoda за миллиард крона ( миллион евро) как сообщ

In [110]:
with open('data.train_text_title.txt', 'w+') as outfile:
    for i in range(len(X_train)):
        outfile.write('__label__' + y_train[i] + ' '+ X_train[i] + '\n')

with open('test.txt', 'w+') as outfile:
    for i in range(len(X_test)):
        outfile.write('__label__' + y_test[i] + ' ' + X_test[i] + '\n')

KeyError: 0

In [105]:
type(X_train)

numpy.ndarray

In [57]:
X = X.tolist()

In [56]:
X

array([['с январь год всё телеканал быть оплачивать услуга передача телерадиосигнал по единый тариф как сообщить министерство российский федерация по дело печать телерадиовещание и средство массовый коммуникация основа для такой решение являться соответствовать постановление правительство в настоящий время общенациональный телеканал (орт нтв вгтрк) оплачивать передача сигнал по льготный тариф который в раз маленький чем у некоторый коммерческий телерадиовещатель с январь для орт нтв и вгтрк тариф быть увеличить в раз для остальной телерадиовещатель снизить',
        'телеканал стать вещать по единый тариф'],
       ['германский автопромышленный концерн volkswagen намеренный полностью купить компания skoda как стать сегодня известно из источник в штаб квартира volkswagen в вольфсбург (земля нижний саксония) германский сторона предложить правительство чехиипродать вольфсбург % пакет акция skoda за миллиард крона ( миллион евро) как сообщаться чешский социала демократ пока заблокировать д

In [74]:
points = [4, 14, 17, 22, 26, 29, 33, 35, 35, 38]
rebounds = [1, 4, 4, 5, 8, 7, 5, 6, 9, 11]

#convert lists into a single list
data = []
data.append(points)
data.append(rebounds)

In [88]:
df = pd.DataFrame(data). transpose ()


ValueError: Buffer has wrong number of dimensions (expected 1, got 2)

In [76]:
df

,0,1
0,4,1
1,14,4
2,17,4
3,22,5
4,26,8
5,29,7
6,33,5
7,35,6
8,35,9
9,38,11


In [87]:
type(points)

list

In [78]:
len(data)

2

In [89]:
len(X)

2

In [90]:
X_text

['с январь год всё телеканал быть оплачивать услуга передача телерадиосигнал по единый тариф как сообщить министерство российский федерация по дело печать телерадиовещание и средство массовый коммуникация основа для такой решение являться соответствовать постановление правительство в настоящий время общенациональный телеканал (орт нтв вгтрк) оплачивать передача сигнал по льготный тариф который в раз маленький чем у некоторый коммерческий телерадиовещатель с январь для орт нтв и вгтрк тариф быть увеличить в раз для остальной телерадиовещатель снизить',
 'германский автопромышленный концерн volkswagen намеренный полностью купить компания skoda как стать сегодня известно из источник в штаб квартира volkswagen в вольфсбург (земля нижний саксония) германский сторона предложить правительство чехиипродать вольфсбург % пакет акция skoda за миллиард крона ( миллион евро) как сообщаться чешский социала демократ пока заблокировать данный сделка считать коммерческий предложение volkswagen явно зан